# Trade Network Analysis
`09_panel.ipynb`

Requires:
1. `imports_decomp.csv` from `08_import_decomp.ipynb`
2. `imf_data.csv` from `04_imfdata.ipynb`

Runs an OLS regression on the panel data above, with time fixed effects. The estimated equation is:

$$ \Delta \text{ln} X_{i,t} = \delta_{t} + \beta_1 \Delta \text{ln} X_{i,t-1} + \beta_2 \Delta q_{i,t-1} + \beta_3 m^{cs}_{i,t-1} + \beta_4 m^{ns}_{i,t-1} + \beta_5 m^{cn}_{i,t-1} + \varepsilon_{i,t} $$ 

---

METODO:
1. Improve description
2. Review for errors!
3. Statsmodels instead of pandas!

#### Libraries

In [22]:
import pandas as pd                                         # pandas dataframes used for convenience
from pandas.stats.plm import PanelOLS
import numpy as np
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

Load IMF data

In [23]:
decomp = pd.read_csv('imports_decomp.csv')
decomp = decomp.rename(columns={'Unnamed: 0': 'date', 'j': 'iso2'})
decomp['date'] = decomp['date'].apply(str)
decomp = decomp.set_index(['date', 'iso2'])
df = pd.read_csv('imf_data.csv', index_col=[0,1,2,3], skipinitialspace=True).reset_index()[2:]
df = df.rename(columns={'level_0': 'iso2', 'Unnamed: 1': 'full_name', 'Unnamed: 2':'date', 'mp': 'mp_base', 
                        'mp.1': 'mp',  'xp': 'xp_base', 'xp.1': 'xp'})
df = df.set_index(['date', 'iso2'])
panel = df.join(decomp)
for v in ['q', 'x', 'xp', 'mp']:
    panel[v] = panel[v].apply(float) # replace country names

#### Deflate USD values with price indices

In [24]:
deflators = {'x': 'xp', 'cs': 'mp', 'ns': 'mp', 'cn': 'mp', 'nn': 'mp'}
for k, v in deflators.iteritems():
    panel[k+'_def'] = np.log(panel[k] * (panel[v] / 100))
    panel[k+'_ndef'] = panel[k] * (panel[v] / 100)
panel['q_ch'] = panel['q'].groupby(df.index.get_level_values(1)).pct_change()

In [25]:
panel['x_ch'] = panel['x_def'].groupby(df.index.get_level_values(1)).diff(periods=1)
panel = panel#.dropna()

# Drop unbalanced countries
obs_size = pd.DataFrame(panel.groupby('full_name').size())
panel = panel.join(obs_size, on='full_name')
panel = panel[panel[0] == panel[0].max()]

country_list = pd.read_csv('oil_temp/c_list.csv', header=None)[0].values
panel2 = panel.reset_index()
panel2 = panel2[panel2['iso2'].isin(country_list)]

panel.to_csv('panel.csv')
panel2.to_csv('panel2.csv')

In [5]:
#reg  = PanelOLS(y=panel['x_ch'],x=panel[['x_def', 'q', 'cs_def', 'ns_def', 'cn_def']],time_effects=True)
#reg